In [1]:
import os
import re
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
import pandas as pd
import pickle
from dgstools.endring import Endring

In [2]:
def save_pickle(obj, fname):
    try:os.mkdir('obj/')
    except:pass
    with open('obj/'+fname + '.pk', 'wb+') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_pickle(fname):
    with open('obj/' + fname + '.pk', 'rb') as f:
        return pickle.load(f)

def write_csv(bag,brand,additional_info='_'):
    import csv
    import random
    import pandas as pd
    try:os.mkdir('comments_AMAZON/')
    except:pass
    df = pd.DataFrame.from_dict(bag, orient='index').T.to_csv('comments_AMAZON/'+brand+str(random.randint(1,1000000))+additional_info+'.csv',index=False)
    print('CSV IS WRITTEN')

def star_finder(driver,STARS_LIST):
    for STAR in driver.find_elements_by_css_selector('.a-link-normal'):
        a = re.search('(.+) out.*',STAR.get_attribute('title'))
        if a:
            STARS_LIST.append(a.group()[0])
        else:pass
    return STARS_LIST

In [3]:
def get_phone_urls_prices(driver,brand):
    url_list=list()
    name_price = dict()
    # How many catalog pages are there?
    try:catalog_pages = int(driver.find_elements(By.CSS_SELECTOR,'li.a-disabled')[-1].text)
    except:
        print("Couldn't find the last page info.")
        try:catalog_pages = int(driver.find_elements_by_class_name('a-normal')[-1].text)
        except:catalog_pages = 2
    
    # Move onto next catalog page.
    print("Number of catalog pages: %s" % catalog_pages)
    for page_nu in range(1,catalog_pages):
        # Next Catalog Page
        # If first page is done, continue to other pages
        
        # GET PRICES
        print("# Searching Phone Names and Prices in the Page %s" % page_nu)
        """
        html = driver.page_source
        soup = BeautifulSoup(html, "lxml")
        
        #We can get price info during comment extraction
        #dirtylist = soup.find('li',{'id':re.compile('result_\d')}).find_all_next(string=True)
        biglist = list(dirtylist)
        phones = [i for i, e in enumerate(biglist) if e == brand]
        for n in phones:
            phonename = biglist[n-2]
            phoneprice = biglist[n+1]
            name_price[phonename]=phoneprice
            print("phonename: {},\nPhoneprice {}".format(phonename, phoneprice))"""

        # GET URLS
        for element in driver.find_elements_by_css_selector('h2.a-size-mini.a-spacing-none.a-color-base.s-line-clamp-2'):
            URL = element.find_element(By.CSS_SELECTOR,"a.a-link-normal.a-text-normal").get_attribute('href')
            url_list.append(URL)
            print(URL) #pRint url
        
        # Next Page
        try:
            #next_url = driver.find_element(By.CLASS_NAME,"a-last").get_attribute('href')
            #driver.get(next_url)
            #Click on Next Button
            driver.find_element(By.CLASS_NAME,"a-last").click()
            print("# Moving on to next catalog page...")
            driver.find_element_by_tag_name('html').send_keys(Keys.END)
            time.sleep(10)
        except: print('There is no other phone page.')

    print("Url gathering process has finished.")
    print("Number of Urls gathered: %s" % len(url_list))
    return url_list,name_price

In [4]:
def text_n_stars():
    text_list=list()
    phone_names = list()
    stars_list = list()
    try:
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR,'.a-section.review')))
        COMMENT_ELEMENT = driver.find_elements_by_css_selector('.a-section.review')
        #  TEXT: Name, Comment, Comment Title, Date, Verified_Purchase
        for person in COMMENT_ELEMENT:
            text = person.text
            text_list.append(text)
            # Add Phone Name
            phone_names.append(driver.find_elements_by_class_name('a-link-normal')[0].text)
            # Add Phone Prices
            #phone_price.append(price)
                        
        # Comment Stars
        star_list = driver.find_element_by_id('cm_cr-review_list').find_elements_by_class_name("a-link-normal")
        for STAR in star_list:
            #print(STAR)
            star_line = STAR.get_attribute('title')
            a = re.search('(.+) out.*',star_line)
            if a:stars_list.append(a.group()[0])
            else:pass
    except:print('WARNING:There was no comments to be seen!') 
    
    return text_list, phone_names, stars_list

In [5]:
#path_to_extension = r"C:\Users\dogus\Dropbox\DgsPy_DBOX\webdrivers\1.16.14_0"
path = ".../chromedriver.exe"

chrome_options = Options()
#chrome_options.add_argument('load-extension=' + path_to_extension)
#chrome_options.add_argument("--headless")

driver = webdriver.Chrome(executable_path=path,chrome_options=chrome_options)
#driver.create_options()

main_url ="https://www.amazon.com/gp/search/other/ref=lp_7072561011_sa_p_89?rh=n%3A2335752011%2Cn%3A%212335753011%2Cn%3A7072561011&bbn=7072561011&pickerToList=lbr_brands_browse-bin&ie=UTF8&qid=1532365672"
driver.get(main_url)

wait = WebDriverWait(driver,20)

In [5]:
driver.quit()

NameError: name 'driver' is not defined

brand_list = ['Alcatel','Alcatel One Touch','Apple','Asus','BlackBerry','Blackview','BLU','BlueCosmo','Casio','CAT PHONES','CNPGD','CUBOT',
'DOOGEE', 'Easyfone', 'FreedomPop', 'Google','GreatCall','H2O Wireless','Honor','HTC','Huawei', 'Jethro','Kyocera','Leeco','LeEco', 'LG', 
'Microsoft','mophie','Motorola','Newmind','Nokia','NUU','OnePlus','Pantech','Plum', 'Razer', 'RCA','Samsung','Seneo','Simple Mobile', 
'Sony','Tile','Total Wireless','Unihertz','Xiaomi', 'ZTE', 'ZTE USA']

In [10]:
gather_links = True
def main():
    brand_list = [
        #'BLU',
        #'Blackview',
        'BlackBerry',
        'OnePlus',
        'ZTE'
        #'Sony',
        #'Asus',
        #'Google',
        #'Xiaomi',
        #'HTC',
        #'Huawei',
        #'Samsung',
        #'Apple'
        #'LG'
        ]
    
    for brand in brand_list:
        # Move on to the MAIN catalog
        xpath = '//a[@title="{}"]'.format(brand)
        driver.find_element_by_xpath(xpath).click()
        
        if gather_links:
            # Find Phone Names and Prices in Catalog Page
            urls,name_price =get_phone_urls_prices(driver,brand=brand) ######
            phone_urls = urls
            #write_csv(name_price,brand,additional_info='Prices')
            save_pickle(phone_urls,brand)
        else:
            phone_urls = load_pickle(brand)
            
        # Iterate over phones on pages
        print("# Iterate over phones on pages")
        url_amount = len(phone_urls[-395:])
        for phone_url in phone_urls[-395:]:
            print("Amount of urls left {}".format(url_amount))
            print('Working on the next phone.\n', phone_url)
            phone_code = phone_url.split('/')[5]
            # Go to that specific phone's comment page.
            link = 'https://www.amazon.com/product-reviews/'+phone_code+'/ref=cm_cr_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1'
            
            # Get down below, comments gets loaded on page
            driver.get(link)
            driver.find_element_by_tag_name('html').send_keys(Keys.END)
            time.sleep(10) # Sleep
            # Gather text, phone name, stars
            text_list,phone_names,stars_list = text_n_stars()
            
            #"Showing #-# of # reviews"
            try:
                wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR,'div.a-section.a-spacing-medium')))
                pagetext = driver.find_elements(By.CSS_SELECTOR,'div.a-section.a-spacing-medium')[1].text
                last_page_number = int(round(int(pagetext.split()[-2].replace(",",""))/10,0))
                print("Total number of pages: %d" % (last_page_number))
            except:
                print("Only one page exist.")
                last_page_number=0
            
            #Iterate over pages
            for c_page_nu in range(1,int(last_page_number)+1):
                # If nextbutton exists, continue to next page
                try:
                    wait.until(EC.visibility_of_element_located((By.XPATH,'//li[@class="a-last"]')))
                    next_button = driver.find_element_by_xpath('//li[@class="a-last"]')
                    next_button.click()
                    time.sleep(3)
                    driver.find_element_by_tag_name('html').send_keys(Keys.END)
                    time.sleep(7)
                    
                    # Gather text, phone name, stars
                    tlist,pnames,slist = text_n_stars()
                    text_list += tlist
                    phone_names += pnames
                    stars_list += slist
                    #print('Moving on to the page %s' % c_page_nu)
                except:
                    print('No "Next Page" button')
                print('Mining process of page %s has finished.' % c_page_nu)
            
            # Iteration over one phone has finished, write down to a csv file.
            url_amount -= 1
            a=len(text_list)
            b=len(phone_names)
            c=len(stars_list)
            if (len(text_list)!=0) & (a==c) & (a==b):
                big_dict = {
                    'Text':text_list,
                    'Phone Title':phone_names,
                    'Stars':stars_list,
                    }
                print(len(text_list),len(phone_names),len(stars_list))
                write_csv(big_dict,brand)
            else:
                print('WARNING: Problem occured while creating a dictionary.List lengths:',a,b,c)
                Endring()

        print('Number of {} phones iterated: {}'.format(brand,len(phone_urls)))
        #print('Amout of comments have been gathered for the {}: {}'.format(brand,len(text_list)))
        
        #Process finished for one brand.
        driver.get(main_url)
        
if __name__=='__main__':
    main()

Number of catalog pages: 8
# Searching Phone Names and Prices in the Page 1
https://www.amazon.com/BlackBerry-Unlocked-Android-Smartphone-T-Mobile/dp/B06ZYGDY4P/ref=sr_1_1?fst=as%3Aoff&qid=1556924057&refinements=p_89%3ABlackBerry&rnid=2528832011&s=mobile&sr=1-1
https://www.amazon.com/BlackBerry-Unlocked-Android-Smartphone-T-Mobile/dp/B07D7TJQ83/ref=sr_1_2?fst=as%3Aoff&qid=1556924057&refinements=p_89%3ABlackBerry&rnid=2528832011&s=mobile&sr=1-2
https://www.amazon.com/BlackBerry-KEYone-64GB-BB100-7-International/dp/B073ZLLVQ9/ref=sr_1_3?fst=as%3Aoff&qid=1556924057&refinements=p_89%3ABlackBerry&rnid=2528832011&s=mobile&sr=1-3
https://www.amazon.com/BlackBerry-Unlocked-Smartphone-T-Mobile-Warranty/dp/B07H25Q45S/ref=sr_1_4?fst=as%3Aoff&qid=1556924057&refinements=p_89%3ABlackBerry&rnid=2528832011&s=mobile&sr=1-4
https://www.amazon.com/Blackberry-Unlocked-Quad-Band-Camera-Keyboard/dp/B005SNNE1Q/ref=sr_1_5?fst=as%3Aoff&qid=1556924057&refinements=p_89%3ABlackBerry&rnid=2528832011&s=mobile&sr=1-

https://www.amazon.com/BlackBerry-Unlocked-World-Phone-Keyboard/dp/B002YDZSQM/ref=sr_1_46?fst=as%3Aoff&qid=1556924059&refinements=p_89%3ABlackBerry&rnid=2528832011&s=mobile&sr=1-46
https://www.amazon.com/BlackBerry-Classic-Factory-Unlocked-SQC100-4/dp/B014VM1CYU/ref=sr_1_47?fst=as%3Aoff&qid=1556924059&refinements=p_89%3ABlackBerry&rnid=2528832011&s=mobile&sr=1-47
https://www.amazon.com/Sprint-BLACKBERRY-STYLE-Smartphone-CONTRACT/dp/B0061G7XFC/ref=sr_1_48?fst=as%3Aoff&qid=1556924059&refinements=p_89%3ABlackBerry&rnid=2528832011&s=mobile&sr=1-48
# Moving on to next catalog page...
# Searching Phone Names and Prices in the Page 3
https://www.amazon.com/BlackBerry-Factory-Unlocked-Shadow-STR100-2/dp/B00UH9AMKM/ref=sr_1_49?fst=as%3Aoff&qid=1556924071&refinements=p_89%3ABlackBerry&rnid=2528832011&s=mobile&sr=1-49
https://www.amazon.com/Blackberry-Tour-Unlocked-Phone-Black/dp/B003X26V50/ref=sr_1_50?fst=as%3Aoff&qid=1556924071&refinements=p_89%3ABlackBerry&rnid=2528832011&s=mobile&sr=1-50
http

https://www.amazon.com/BlackBerry-Passport-Factory-Unlocked-Cellphone/dp/B016ARU55G/ref=sr_1_94?fst=as%3Aoff&qid=1556924082&refinements=p_89%3ABlackBerry&rnid=2528832011&s=mobile&sr=1-94
https://www.amazon.com/BlackBerry-9300-Contract-Global-Smartphone/dp/B0092GJ5R2/ref=sr_1_95?fst=as%3Aoff&qid=1556924082&refinements=p_89%3ABlackBerry&rnid=2528832011&s=mobile&sr=1-95
https://www.amazon.com/Blackberry-Curve-9330-3G-SPRINT/dp/B00513N5EO/ref=sr_1_96?fst=as%3Aoff&qid=1556924082&refinements=p_89%3ABlackBerry&rnid=2528832011&s=mobile&sr=1-96
# Moving on to next catalog page...
# Searching Phone Names and Prices in the Page 5
https://www.amazon.com/BlackBerry-Unlocked-Touchscreen-Camera-No-Warranty/dp/B006JI3RKC/ref=sr_1_97?fst=as%3Aoff&qid=1556924094&refinements=p_89%3ABlackBerry&rnid=2528832011&s=mobile&sr=1-97
https://www.amazon.com/BlackBerry-STH100-2-Unlocked-Smartphone-International/dp/B01JD8P7AW/ref=sr_1_98?fst=as%3Aoff&qid=1556924094&refinements=p_89%3ABlackBerry&rnid=2528832011&s=mob

https://www.amazon.com/BlackBerry-Dual-Band-Bluetooth-Smartphone-Keyboard/dp/B004HNLOSI/ref=sr_1_143?fst=as%3Aoff&qid=1556924105&refinements=p_89%3ABlackBerry&rnid=2528832011&s=mobile&sr=1-143
https://www.amazon.com/BlackBerry-Curve-9220-Quad-Band-Bluetooth/dp/B008QVR2DS/ref=sr_1_144?fst=as%3Aoff&qid=1556924105&refinements=p_89%3ABlackBerry&rnid=2528832011&s=mobile&sr=1-144
# Moving on to next catalog page...
# Searching Phone Names and Prices in the Page 7
https://www.amazon.com/Sprint-BlackBerry-Contract-Camera-Smartphone/dp/B005TEJXRS/ref=sr_1_145?fst=as%3Aoff&qid=1556924117&refinements=p_89%3ABlackBerry&rnid=2528832011&s=mobile&sr=1-145
https://www.amazon.com/Blackberry-Unlocked-Phone-QWERTY-Keybaord/dp/B00EQ0BHK2/ref=sr_1_146?fst=as%3Aoff&qid=1556924117&refinements=p_89%3ABlackBerry&rnid=2528832011&s=mobile&sr=1-146
https://www.amazon.com/BlackBerry-Curve-8310-GSM-Unlocked/dp/B005KMK34G/ref=sr_1_147?fst=as%3Aoff&qid=1556924117&refinements=p_89%3ABlackBerry&rnid=2528832011&s=mobile

Total number of pages: 21
Mining process of page 1 has finished.
Mining process of page 2 has finished.
Mining process of page 3 has finished.
Mining process of page 4 has finished.
Mining process of page 5 has finished.
Mining process of page 6 has finished.
Mining process of page 7 has finished.
Mining process of page 8 has finished.
Mining process of page 9 has finished.
Mining process of page 10 has finished.
Mining process of page 11 has finished.
Mining process of page 12 has finished.
Mining process of page 13 has finished.
Mining process of page 14 has finished.
Mining process of page 15 has finished.
Mining process of page 16 has finished.
Mining process of page 17 has finished.
Mining process of page 18 has finished.
Mining process of page 19 has finished.
Mining process of page 20 has finished.
Mining process of page 21 has finished.
212 212 212
CSV IS WRITTEN
Amount of urls left 166
Working on the next phone.
 https://www.amazon.com/BlackBerry-KEYone-64GB-BB100-7-Internatio

...

Total number of pages: 2
Mining process of page 1 has finished.
Mining process of page 2 has finished.
25 25 25
CSV IS WRITTEN
Amount of urls left 40
Working on the next phone.
 https://www.amazon.com/BlackBerry-STJ100-1-Touchscreen-Long-Lasting-Smartphone/dp/B00LNL6Y4Q/ref=sr_1_129?fst=as%3Aoff&qid=1556924105&refinements=p_89%3ABlackBerry&rnid=2528832011&s=mobile&sr=1-129
Total number of pages: 1
No "Next Page" button
Mining process of page 1 has finished.
9 9 9
CSV IS WRITTEN
Amount of urls left 39
Working on the next phone.
 https://www.amazon.com/BLACKBERRY-Q10-Keyboard-Factory-Unlocked/dp/B00CMH9ZIM/ref=sr_1_130?fst=as%3Aoff&qid=1556924105&refinements=p_89%3ABlackBerry&rnid=2528832011&s=mobile&sr=1-130
Total number of pages: 3
Mining process of page 1 has finished.
Mining process of page 2 has finished.
No "Next Page" button
Mining process of page 3 has finished.
26 26 26
CSV IS WRITTEN
Amount of urls left 38
Working on the next phone.
 https://www.amazon.com/Blackberry-Curve-8310

No "Next Page" button
Mining process of page 1 has finished.
9 9 9
CSV IS WRITTEN
Amount of urls left 17
Working on the next phone.
 https://www.amazon.com/Blackberry-SQN100-3-16GB-Unlocked-Phone/dp/B00LW3RN6I/ref=sr_1_152?fst=as%3Aoff&qid=1556924117&refinements=p_89%3ABlackBerry&rnid=2528832011&s=mobile&sr=1-152
Total number of pages: 1
Mining process of page 1 has finished.
11 11 11
CSV IS WRITTEN
Amount of urls left 16
Working on the next phone.
 https://www.amazon.com/BlackBerry-PRD-30903-011-Unlocked-Smartphone-Camera/dp/B003QXLRH8/ref=sr_1_153?fst=as%3Aoff&qid=1556924117&refinements=p_89%3ABlackBerry&rnid=2528832011&s=mobile&sr=1-153
Total number of pages: 3
Mining process of page 1 has finished.
Mining process of page 2 has finished.
Mining process of page 3 has finished.
33 33 33
CSV IS WRITTEN
Amount of urls left 15
Working on the next phone.
 https://www.amazon.com/BlackBerry-Dual-SIM-BBF100-6-Unlocked-Smartphone/dp/B07DNPZ5ZZ/ref=sr_1_154?fst=as%3Aoff&qid=1556924117&refineme

https://www.amazon.com/OnePlus-McLaren-Orange-Storage-Unlocked/dp/B07DJCVTBH/ref=sr_1_20?fst=as%3Aoff&qid=1556946955&refinements=p_89%3AOnePlus&rnid=2528832011&s=mobile&sr=1-20
https://www.amazon.com/OnePlus-A3000-Unlocked-Version-Warranty/dp/B072343XWL/ref=sr_1_21?fst=as%3Aoff&qid=1556946955&refinements=p_89%3AOnePlus&rnid=2528832011&s=mobile&sr=1-21
https://www.amazon.com/OnePlus-64GB-ROM-A3000-Factory-Unlocked/dp/B01MXR13TZ/ref=sr_1_22?fst=as%3Aoff&qid=1556946955&refinements=p_89%3AOnePlus&rnid=2528832011&s=mobile&sr=1-22
https://www.amazon.com/OnePlus-Storage-Factory-Unlocked-Display/dp/B07K79G75L/ref=sr_1_23?fst=as%3Aoff&qid=1556946955&refinements=p_89%3AOnePlus&rnid=2528832011&s=mobile&sr=1-23
https://www.amazon.com/OnePlus-5T-A5010-Version-Midnight/dp/B0785H6VKX/ref=sr_1_24?fst=as%3Aoff&qid=1556946955&refinements=p_89%3AOnePlus&rnid=2528832011&s=mobile&sr=1-24
# Moving on to next catalog page...
Url gathering process has finished.
Number of Urls gathered: 24
# Iterate over phone

Total number of pages: 17
Mining process of page 1 has finished.
Mining process of page 2 has finished.
Mining process of page 3 has finished.
Mining process of page 4 has finished.
Mining process of page 5 has finished.
Mining process of page 6 has finished.
Mining process of page 7 has finished.
Mining process of page 8 has finished.
Mining process of page 9 has finished.
Mining process of page 10 has finished.
Mining process of page 11 has finished.
Mining process of page 12 has finished.
Mining process of page 13 has finished.
Mining process of page 14 has finished.
Mining process of page 15 has finished.
Mining process of page 16 has finished.
No "Next Page" button
Mining process of page 17 has finished.
166 166 166
CSV IS WRITTEN
Amount of urls left 7
Working on the next phone.
 https://www.amazon.com/OnePlus-Factory-Unlocked-Phone-Midnight/dp/B07D7Q9PPG/ref=sr_1_18?fst=as%3Aoff&qid=1556946955&refinements=p_89%3AOnePlus&rnid=2528832011&s=mobile&sr=1-18
Total number of pages: 0
1 

Total number of pages: 0
3 3 3
CSV IS WRITTEN
Amount of urls left 5
Working on the next phone.
 https://www.amazon.com/ZTE-Z831-Prepaid-Carrier-Locked/dp/B06XVRX1NR/ref=sr_1_6?fst=as%3Aoff&qid=1556948407&refinements=p_89%3AZTE&rnid=2528832011&s=mobile&sr=1-6
Total number of pages: 0
1 1 1
CSV IS WRITTEN
Amount of urls left 4
Working on the next phone.
 https://www.amazon.com/dp/B07RH8DZRM/ref=sr_1_7?fst=as%3Aoff&qid=1556948407&refinements=p_89%3AZTE&rnid=2528832011&s=mobile&sr=1-7
Only one page exist.
Amount of urls left 3
Working on the next phone.
 https://www.amazon.com/ZTE-Gaming-Smartphone-Fingerprint-Android/dp/B07PW3T4YH/ref=sr_1_8?fst=as%3Aoff&qid=1556948407&refinements=p_89%3AZTE&rnid=2528832011&s=mobile&sr=1-8
Only one page exist.
Amount of urls left 2
Working on the next phone.
 https://www.amazon.com/ZTE-Axon-7-grey-unlocked/dp/B01I092IPC/ref=sr_1_9?fst=as%3Aoff&qid=1556948407&refinements=p_89%3AZTE&rnid=2528832011&s=mobile&sr=1-9
Only one page exist.
Amount of urls left 1
